In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import cosine
from sklearn import svm

In [2]:
train_set_path = "/home/weizhuozhang/Desktop/cwb/COMP90051-Project1/data/BERT_encoding/train_encode_tuned.npy"
dev_set_path = "/home/weizhuozhang/Desktop/cwb/COMP90051-Project1/data/BERT_encoding/dev_encode_tuned.npy"
train_set_csv = "/home/weizhuozhang/Desktop/cwb/COMP90051-Project1/data/train_set_v1.txt"
dev_set_csv = "/home/weizhuozhang/Desktop/cwb/COMP90051-Project1/data/dev_set_v1.txt"

train_set = np.load(train_set_path)
train_csv = pd.read_csv(train_set_csv, sep='\t', header=None)
train_csv = np.array(train_csv)
dev_set = np.load(dev_set_path)
dev_csv = pd.read_csv(dev_set_csv, sep='\t', header=None)
dev_csv = np.array(dev_csv)

train_label = (train_csv[:,0]).astype('int')
dev_label = (dev_csv[:,0]).astype('int')

In [ ]:
import time
k_neighbors = [3, 7, 9, 13, 15]
for k in k_neighbors:
    knn_clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=k)
    knn_clf.fit(train_set, train_label)
    time_start=time.time()
    predicted = knn_clf.predict(dev_set)
    accuracy = sum(predicted == dev_label)/len(predicted)
    time_end=time.time()
    print("---------------------------------")
    print("knn k={0}".format(k))
    print("accuracy: {0}".format(accuracy))
    print("Time spent: {0:.2f}ms".format((time_end-time_start)*1000))


In [ ]:
%%time
k = 13
knn_clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=k, metric=cosine)
knn_clf.fit(train_set, train_label)
predicted = knn_clf.predict(dev_set)
accuracy = sum(predicted == dev_label)/len(predicted)
print(accuracy)

In [ ]:
%%time
k = 20
knn_clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=k, metric=cosine)
knn_clf.fit(train_set, train_label)

k_neighbors_list = knn_clf.kneighbors(dev_set[:10])

In [ ]:
%%time
k = 1000
knn_clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=k)
knn_clf.fit(train_set, train_label)

k_neighbors_list = knn_clf.kneighbors(dev_set)
k_neigbbors_path = "/home/weizhuozhang/workspace/COMP90051-Project1/data/result/k_neighbors_list_k_eq_{0}".format(k)
np.save(k_neigbbors_path, k_neighbors_list)

In [ ]:
k_neigbbors_npy_path = "/home/weizhuozhang/workspace/COMP90051-Project1/data/result/k_neighbors_list_k_eq_{0}.npy".format(k)
k_neighbors_list = np.load(k_neigbbors_npy_path)
k_neighbors_dist_list = k_neighbors_list[0]
k_neighbors_index_list = k_neighbors_list[1]
label_k_neighbors_np = []
for i, index_list in enumerate(k_neighbors_index_list):
    line = []
    line.append(dev_label[i])
    candidate = list(train_label[index_list.astype('int')])
    line.append(candidate)
    label_k_neighbors_np.append(line)
label_k_neighbors_np = np.array(label_k_neighbors_np)

In [56]:
true_label_list = label_k_neighbors_np[:,0]
candidate_label_list = label_k_neighbors_np[:,1]
counter = 0
for i, label in enumerate(true_label_list):
    if label in candidate_label_list[i]:
        counter += 1
candidate_accuracy = counter / len(true_label_list)
print(candidate_accuracy)

0.7219577012512457


In [57]:
counter = 0
for i, label in enumerate(true_label_list):
    counter += len(np.unique(candidate_label_list[i]))
avg_user_counter = counter / len(candidate_label_list)
print(avg_user_counter)

/anaconda3/envs/sml/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]